In [36]:
# imports
import pandas as pd
import numpy as np
import itertools
import random
from geopy.distance import geodesic

In [37]:
# load the cities dataset
MHQoL = pd.read_csv('/content/data/movehubqualityoflife.csv')
cities = pd.read_csv('/content/data/cities.csv')
HappinessIndex = pd.read_csv('/content/data/2019.csv')
UnescoSites = pd.read_csv('/content/data/whc-sites-2019.csv')
MHQoL.head()
cities.head()
HappinessIndex.head()
UnescoSites.head()

,category,states_name_en,region_en,unique_number,id_no,rev_bis,name_en,short_description_en,justification_en,date_inscribed,secondary_dates,danger,date_end,danger_list,longitude,latitude,area_hectares,criteria_txt,category_short,iso_code,udnp_code,transboundary
0,Cultural,Afghanistan,Asia and the Pacific,230,208,Rev,Cultural Landscape and Archaeological Remains ...,The cultural landscape and archaeological rem...,<em>Criterion (i):</em> The Buddha statues an...,2003,NaN,1,NaN,Y 2003,67.825250,34.846940,158.9265,(i)(ii)(iii)(iv)(vi),C,af,afg,0
1,Cultural,Afghanistan,Asia and the Pacific,234,211,Rev,Minaret and Archaeological Remains of Jam,"The 65m-tall Minaret of Jam is a graceful, so...",<em>Criterion (ii):</em> The innovative archi...,2002,NaN,1,NaN,Y 2002,64.515889,34.396417,70.0000,(ii)(iii)(iv),C,af,afg,0
2,Cultural,Albania,Europe and North America,1590,569,Bis,Historic Centres of Berat and Gjirokastra,Berat and Gjirokastra are inscribed as rare e...,NaN,2005,2008,0,NaN,NaN,20.133333,40.069444,58.9000,(iii)(iv),C,al,alb,0
3,Cultural,Albania,Europe and North America,1563,570,ter,Butrint,"Inhabited since prehistoric times, Butrint ha...",NaN,1992,1999,0,2005.0,P 1997-2005,20.026111,39.751111,NaN,(iii),C,al,alb,0
4,Cultural,Algeria,Arab States,111,102,NaN,Al Qal'a of Beni Hammad,In a mountainous site of extraordinary beauty...,NaN,1980,NaN,0,NaN,NaN,4.786840,35.818440,150.0000,(iii),C,dz,dza,0


In [62]:
def function(city):
  distances= UnescoSites.apply(lambda sites : geodesic((sites['latitude'],sites['longitude']), (city['lat'], city['lng'])).kilometers,1)
  return sum(map(lambda x : x<100, distances))


In [70]:
City_Country = pd.merge(MHQoL[['City','Purchase Power', 'Health Care', 'Pollution', 'Quality of Life', 'Crime Rating','lat','lng']],
                 cities,
                 on='City')

HappinessIndex=HappinessIndex.rename(columns={'Country or region': 'Country', 'Score': 'Happiness_Score'})

City_Country_Happiness = pd.merge(City_Country,
                 HappinessIndex[['Country','Happiness_Score']],
                 on='Country')

City_Country_Happiness['Unesco']=City_Country_Happiness.apply(function, 1)


City_Country_Happiness.head(20)



,City,Purchase Power,Health Care,Pollution,Quality of Life,Crime Rating,lat,lng,Country,Happiness_Score,Unesco
0,Caracas,11.25,44.44,83.45,8.61,85.70,10.480594,-66.903606,Venezuela,4.707,1
1,Barcelona,45.68,58.47,71.75,47.18,41.47,36.088106,-95.924131,Venezuela,4.707,0
2,Valencia,49.11,72.07,35.24,64.89,26.04,28.521792,-81.463577,Venezuela,4.707,0
3,Johannesburg,53.99,59.98,47.39,51.26,83.93,-26.204103,28.047305,South Africa,4.722,0
4,Cape Town,60.36,71.67,75.98,78.73,68.06,-33.924869,18.424055,South Africa,4.722,2
5,Pretoria,46.74,71.11,70.13,61.44,68.06,-25.747868,28.229271,South Africa,4.722,0
6,Fortaleza,52.28,45.46,66.32,36.68,78.65,-3.732714,-38.526998,Brazil,6.300,0
7,Porto Alegre,19.07,51.01,86.16,31.87,76.46,-30.034647,-51.217658,Brazil,6.300,0
8,Belo Horizonte,21.77,57.26,59.73,36.26,50.99,-19.916681,-43.934493,Brazil,6.300,3
9,Colombo,20.18,75.00,33.59,26.68,50.00,6.927079,79.861243,Brazil,6.300,2


In [72]:
features = ['Purchase Power', 'Health Care', 'Quality of Life', 'Pollution', 'Crime Rating', 'Happiness_Score', 'Unesco']

norm = lambda xs: (xs-xs.min())/(xs.max()-xs.min())
City_Country_Happiness[features] = (norm(City_Country_Happiness[features]) * 100)

City_Country_Happiness['QoL_H']=(City_Country_Happiness['Quality of Life']+City_Country_Happiness['Happiness_Score'])/2

City_Country_Happiness.head(20)

,City,Purchase Power,Health Care,Pollution,Quality of Life,Crime Rating,lat,lng,Country,Happiness_Score,Unesco,QoL_H
0,Caracas,5.697906,31.425529,90.294309,3.783045,100.000000,10.480594,-66.903606,Venezuela,28.474655,9.090909,16.128850
1,Barcelona,45.981046,50.099827,77.634711,47.732452,40.900588,36.088106,-95.924131,Venezuela,28.474655,0.000000,38.103554
2,Valencia,49.994150,68.201784,38.130275,67.912489,20.283271,28.521792,-81.463577,Venezuela,28.474655,0.000000,48.193572
3,Johannesburg,55.703756,52.109677,51.276780,52.381495,97.634955,-26.204103,28.047305,South Africa,28.825041,0.000000,40.603268
4,Cape Town,63.156663,67.669373,82.211643,83.682771,76.429717,-33.924869,18.424055,South Africa,28.825041,18.181818,56.253906
5,Pretoria,47.221247,66.923998,75.881844,63.981313,76.429717,-25.747868,28.229271,South Africa,28.825041,0.000000,46.403177
6,Fortaleza,53.703054,32.783176,71.759359,35.768004,90.579904,-3.732714,-38.526998,Brazil,65.685587,0.000000,50.726796
7,Porto Alegre,14.847315,40.170371,93.226574,30.287147,87.653661,-30.034647,-51.217658,Brazil,65.685587,0.000000,47.986367
8,Belo Horizonte,18.006318,48.489285,64.628868,35.289426,53.621058,-19.916681,-43.934493,Brazil,65.685587,27.272727,50.487507
9,Colombo,16.146016,72.101690,36.344947,24.373291,52.298236,6.927079,79.861243,Brazil,65.685587,18.181818,45.029439


In [4]:
def generate_dataset(multiplier = 5):
    
    # 0 is none, 1 is low, 2 is mid, 3 is high
    choices = [[0, 1, 2, 3], [0, 1, 2, 3], [0, 1, 2, 3], [0, 1, 2, 3], [0, 1, 2, 3]]
    
    # create all possible combinations and repeat them to create bigger ds
    combinations = list(itertools.product(*choices)) * multiplier
    
    dataset = pd.DataFrame(columns=['Weights', 'City'])
    dataset['Weights'] = pd.Series(combinations)
    
    for index, row in dataset.iterrows():
        row['City'] = get_city(np.array(row['Weights']))
        
    return dataset

In [5]:
def get_city(weights):
    
    # rank cities according to the weights given by the person
    ranked_cities = rank_cities(weights).reset_index(drop=True)
    
    # sample from uniform distribution and "randomly" select a city
    sample = np.random.uniform(0, 1, len(ranked_cities))
    prob_score = sample * list(ranked_cities['Score'])
    maximum = max(prob_score)
    if maximum != 0:
        chosen = [int(i/maximum) for i in prob_score]
    else:
        chosen = [0] * len(ranked_cities)
        chosen[random.randint(0, len(chosen))] = 1
        
    ranked_cities['Chosen'] = pd.Series(chosen)
        
    return ranked_cities[ranked_cities['Chosen'] == 1]['City'].values[0]

In [6]:
def rank_cities(weights):
    features = ['Purchase Power', 'Health Care', 'Quality of Life', 'Pollution', 'Crime Rating']
    
    # pollution and crime rating have a negative impact, whist the other features have a positive one
    weights *= [1, 1, 1, -1, -1]
    
    norm = lambda xs: (xs-xs.min())/(xs.max()-xs.min())
    # e^(2x) to increase the probability of getting the most compatible city
    cities['Score'] = np.exp(norm(cities[features].dot(weights)) * 10)
    
    return cities.sort_values('Score', ascending=False).fillna(0)

In [7]:
people = generate_dataset(5)

In [8]:
people

,Weights,City
0,"(0, 0, 0, 0, 0)",Budapest
1,"(0, 0, 0, 0, 1)",Abu Dhabi
2,"(0, 0, 0, 0, 2)",Marbella
3,"(0, 0, 0, 0, 3)",Malaga
4,"(0, 0, 0, 1, 0)",Nicosia
...,...,...
5115,"(3, 3, 3, 2, 3)",Dresden
5116,"(3, 3, 3, 3, 0)",Glasgow
5117,"(3, 3, 3, 3, 1)",Glasgow
5118,"(3, 3, 3, 3, 2)",Zurich
